<a href="https://colab.research.google.com/github/otanet/hannari_NLP_hands_on_2022/blob/main/4_named_entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GiNZAのインストール
!pip install ginza==4.0.5

# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('ja_ginza')
doc = nlp('山田さんと銀座でランチをご一緒しましょう。')

# 固有表現抽出
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char)) # 終了位置

# 固有表現抽出の強調表示
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
import spacy
nlp = spacy.load('ja_ginza')

# 固有表現抽出
doc = nlp('サツキと妹のメイは、母の療養のために父と一緒に農村へ引っ越してきた。')
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char) # 終了位置
    )

In [ ]:
import spacy
nlp = spacy.load('ja_ginza')

# 固有表現ルールの追加
from spacy.pipeline import EntityRuler
ruler = EntityRuler(nlp)
ruler.add_patterns([
    {'label': 'Person', 'pattern': 'サツキ'},
    {'label': 'Person', 'pattern': 'メイ'}])
nlp.add_pipe(ruler)

# 固有表現抽出
doc = nlp('サツキと妹のメイは、母の療養のために父と一緒に農村へ引っ越してきた。')
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char)) # 終了位置

In [ ]:
import spacy
import random

# 固有抽出表現モデルの学習の関数定義
def train_ner(train_data, epoch):
    # 日本語の空モデルの生成
    nlp = spacy.blank('ja')

    # 固有表現抽出モデルのパイプの追加
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # ラベルの追加
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # 固有表現抽出モデルのみ学習
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
       
        # 学習ループ
        for itn in range(epoch):
            # シャッフル
            random.shuffle(train_data)
           
            # 学習
            losses = {}
            for text, annotations in train_data:
                nlp.update([text], [annotations], drop=0.2, sgd=optimizer, losses=losses)
            print('iteration'+str(itn)+': '+str(losses['ner']))
    return nlp

# 学習データの準備
train_data = [
    ('入院している母のお見舞いに行ったサツキとメイはオバケ屋敷のことを報告した。', 
        {'entities': [(16, 19, 'Person'),(20, 22, 'Person')]}),
    ('サツキとメイが森のバス停で雨の中父の帰りを待っている。', 
        {'entities': [(0, 3, 'Person'), (4, 6, 'Person')]}),
    ('1人で遊んでいたメイは庭で不思議な生き物を見つけた。', 
        {'entities': [(8, 10, 'Person')]}),
    ('人が住み始めるといつのまにか居なくなるという話を聞いてサツキは拍子抜けした。', 
        {'entities': [(27, 30, 'Person')]})
]

# 固有表現抽出モデルの学習
nlp = train_ner(train_data, 100)

# 固有表現抽出モデルの保存
nlp.to_disk('ner_model')

In [ ]:
import spacy

# 固有表現抽出モデルの読み込み
nlp = spacy.load('ner_model')

# 固有表現抽出
doc = nlp('サツキと妹のメイは、母の療養のために父と一緒に農村へ引っ越してきた。')
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char) # 終了位置
    )

In [ ]:
%%time
import spacy
import random

# 固有抽出表現モデルの学習の関数定義
def train_ner(train_data, epoch):
    # 日本語の空モデルの生成
    nlp = spacy.blank('ja')

    # 固有表現抽出モデルのパイプの追加
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # ラベルの追加
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # 固有表現抽出モデルのみ学習
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
       
        # 学習ループ
        for itn in range(epoch):
            # シャッフル
            random.shuffle(train_data)
           
            # 学習
            losses = {}
            for text, annotations in train_data:
                nlp.update([text], [annotations], drop=0.2, sgd=optimizer, losses=losses)
            print('iteration{}: {:.8f}'.format(itn, losses['ner']))
    return nlp

# 学習データの準備
train_data = [
    ('入院している母のお見舞いに行ったサツキとメイはオバケ屋敷のことを報告した。', 
        {'entities': [(16, 19, 'Person'),(20, 22, 'Person')]}),
    ('サツキとメイが森のバス停で雨の中父の帰りを待っている。', 
        {'entities': [(0, 3, 'Person'), (4, 6, 'Person')]}),
    ('1人で遊んでいたメイは庭で不思議な生き物を見つけた。', 
        {'entities': [(8, 10, 'Person')]}),
    ('人が住み始めるといつのまにか居なくなるという話を聞いてサツキは拍子抜けした。', 
        {'entities': [(27, 30, 'Person')]})
]

# 固有表現抽出モデルの学習
nlp = train_ner(train_data, 50)

# 固有表現抽出モデルの保存
nlp.to_disk('ner_model')

In [ ]:
import spacy

# 固有表現抽出モデルの読み込み
nlp = spacy.load('ner_model')

# 固有表現抽出
doc = nlp('サツキと妹のメイは、母の療養のために父と一緒に農村へ引っ越してきた。')
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char) # 終了位置
    )

In [ ]:
%%time
import spacy
import random

# 固有抽出表現モデルの学習の関数定義
def train_ner(train_data, epoch):
    # 日本語の空モデルの生成
    nlp = spacy.blank('ja')

    # 固有表現抽出モデルのパイプの追加
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # ラベルの追加
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # 固有表現抽出モデルのみ学習
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
       
        # 学習ループ
        for itn in range(epoch):
            # シャッフル
            random.shuffle(train_data)
           
            # 学習
            losses = {}
            for text, annotations in train_data:
                nlp.update([text], [annotations], drop=0.2, sgd=optimizer, losses=losses)
            print('iteration{}: {:.8f}'.format(itn, losses['ner']))
    return nlp

# 学習データの準備
import json
labels = {
    '人名': 'Person',
    '法人名': 'Juridical_Person',
    '政治的組織名': 'Political_Organization',
    'その他の組織名': 'Organization_Other',
    '地名': 'Location',
    '施設名': 'Facility',
    '製品名': 'Product',
    'イベント名': 'Event',
}
json_data = json.load(open('ner.json', 'r'))
train_data = []
for data in json_data:
    text = data['text']
    entities = data['entities']
    value = []
    for entity in entities:
        span = entity['span']
        label = labels[entity['type']]
        value.append((span[0], span[1], label))
    train_data.append((text, {'entities': value}))

# 固有表現抽出モデルの学習
nlp = train_ner(train_data, 50)

# 固有表現抽出モデルの保存
nlp.to_disk('ner_model')

In [ ]:
import spacy

# 固有表現抽出モデルの読み込み
nlp = spacy.load('ner_model')

# 固有表現抽出
doc = nlp('サツキと妹のメイは、母の療養のために父と一緒に農村へ引っ越してきた。')
for ent in doc.ents:
    print(
        ent.text+','+ # テキスト
        ent.label_+','+ # ラベル
        str(ent.start_char)+','+ # 開始位置
        str(ent.end_char) # 終了位置
    )